In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

### LangGraph

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages  # 매 단계마다 새로운 메시지를 추가함
from langchain_core.messages import AnyMessage # HumanMessage, AIMessage, SystemMessage, ToolMessage, FunctionMessage를 나타내는 추상화된 타입

# 상태 객체 생성용 클래스
# 딕셔너리 구조
class AgentState(TypedDict):
    messages: list[Annotated[AnyMessage, add_messages]] # add_messages 리듀서 함수는 상태 키의 값(메시지)을 업데이트(덮어쓰는 대신 리스트에 추가)

In [4]:
from langgraph.graph import StateGraph # 상태 기반의 그래프를 생성하는 클래스(에이전트의 상태 관리, Node와 Edge 연결)

# 그래프 객체
graph_builder = StateGraph(AgentState)

In [8]:
# 함수 노드
def generate(state: AgentState) -> AgentState:
    return {'messages': llm.invoke(state["messages"])}

In [9]:
# 그래프 객체에 노드 추가
graph_builder.add_node('generate', generate)

In [10]:
from langgraph.graph import START, END # 그래프의 시작점과 끝점을 명시적으로 정의하는 데 사용되는 특별한 상수

# 엣지로 노드 연결
graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

In [11]:
graph = graph_builder.compile() # 그래프를 실행 가능한 상태로 변환하는 메서드

In [12]:
# uv add grandalf
graph.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [13]:
from langchain_core.messages import HumanMessage

initial_state = {'messages': [HumanMessage("내 이름은 김일남이야")]}
graph.invoke(initial_state)

{'messages': AIMessage(content='네, 김일남 님이시군요. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2d3ad269-15e8-4e50-8be4-5eddd4f57b6f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 1127, 'total_tokens': 1135, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1113}})}

In [ ]:
initial_state = {'messages': [HumanMessage("내 이름은?")]} # 멀티턴 아님!
graph.invoke(initial_state)

{'messages': AIMessage(content='저는 사용자님의 이름을 알 수 없습니다. 저는 개인 정보를 저장하지 않기 때문입니다.\n\n혹시 저에게 알려주시면, 이 대화 동안은 기억하고 사용할 수 있습니다. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--64e46215-914a-4f69-a12e-e2cdc5774c10-0', usage_metadata={'input_tokens': 5, 'output_tokens': 681, 'total_tokens': 686, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 642}})}

In [ ]:
while True:
    user_input = input("You: ")

    if user_input == "종료":
        break

    for event in graph.stream({'messages': [HumanMessage(user_input)]}):
        content_value = event['generate']['messages'].content
        print("AI:", content_value)  

AI: 네, 김일남 님이시군요. 만나서 반갑습니다!
AI: 저는 인공지능이기 때문에 당신의 이름을 알 수 없습니다. 저는 개인 정보를 가지고 있지 않아요.

만약 저에게 이름을 알려주신다면, 이 대화 동안은 기억하고 사용할 수 있습니다!
